In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CSV_FOLDER = "/content/drive/MyDrive/hea_datasets"
FAISS_INDEX_PATH = "/content/drive/MyDrive/faiss_index_hugface_BAAI_new"

In [ ]:
#load csvs
import pandas as pd

d1 = pd.read_csv(f"{CSV_FOLDER}/dataset1_clean.csv")
d2 = pd.read_csv(f"{CSV_FOLDER}/dataset2_clean.csv")
d3 = pd.read_csv(f"{CSV_FOLDER}/dataset3_clean.csv")

print("CSVs loaded")
print("Dataset 1 rows:", len(d1), "columns:", d1.columns.tolist())
print("Dataset 2 rows:", len(d2), "columns:", d2.columns.tolist())
print("Dataset 3 rows:", len(d3), "columns:", d3.columns.tolist())

CSVs loaded
Dataset 1 rows: 1545 columns: ['id', 'formula', 'microstructure', 'processing', 'phase_label', 'grain_size', 'density_exp', 'density_calc', 'HV', 'PROPERTY: Type of test', 'PROPERTY: Test temperature ($^\\circ$C)', 'YS', 'UTS', 'elongation', 'PROPERTY: Elongation plastic (%)', 'modulus_exp', 'modulus_calc', 'PROPERTY: O content (wppm)', 'PROPERTY: N content (wppm)', 'PROPERTY: C content (wppm)', 'doi', 'year', 'title', 'composition_norm']
Dataset 2 rows: 1361 columns: ['id', 'formula', 'density_calc', 'Hmix', 'Smix', 'Gmix', 'Tm', 'delta', 'elec_diff', 'VEC', 'processing', 'microstructure', 'doi', 'composition_norm']
Dataset 3 rows: 1117 columns: ['formula', 'No', 'S_Phase', 'Phase', 'delta', 'Tm', 'D_Tm (K)', '?Hmix (kJ/mol)', 'elec_diff', 'VEC', 'modulus_calc', 'D_Bulk (GPa)', 'Tm.1', 'composition_norm']


In [ ]:
!pip install langchain langchain-community faiss-cpu torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
from huggingface_hub import login
login(new_session=False)


In [ ]:
#  Load FAISS vector DB

from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en",
    model_kwargs={"device": device}
)

faiss_index = FAISS.load_local(FAISS_INDEX_PATH, embeddings,allow_dangerous_deserialization=True)
print(" FAISS vector DB loaded")

Using device: cuda


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

 FAISS vector DB loaded


In [ ]:
!pip install -q transformers accelerate

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

llm_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    #max_length=512,
    #temperature=0.2
)

print("✅ Mistral loaded")

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


✅ Mistral loaded


In [ ]:
import re
import pandas as pd

# Property synonyms map
property_synonyms = {
    "hardness": ["hv", "hardness", "vicker's hardness", "vickers hardness"],
    "bulk modulus": ["d_bulk (gpa)", "bulk modulus", "bulk_modulus", "bulk modulus"],
    "yield_strength": ["ys", "yield stress", "yield strength"],
    "ultimate_strength": ["uts", "tensile strength", "ultimate tensile strength"],
    "phase": ["phase_label", "bcc/fcc/other", "phase", "microstructure"],
    "density": ["density_exp", "density_calc", "density"]
}

def find_column_for_property(df, property_name):
    synonyms = property_synonyms.get(property_name.lower(), [property_name])
    for syn in synonyms:
        for col in df.columns:
            if syn.lower() in col.lower():
                return col
    return None

def parse_query_to_filters(question):
    """
    Parse question to a dict of property filters.
    Supports phase filters, numeric comparisons, and high/low keywords.
    """
    query = question.lower()

    filters = {}

    # Detect phase filter (fcc, bcc, hcp, other)
    for phase in ["fcc", "bcc", "hcp", "other"]:
        if phase in query:
            filters["phase"] = phase
            break

    # Detect numeric filters (hardness > 200 etc.)
    numeric_props = ["hardness", "bulk modulus", "yield strength", "ultimate strength", "density"]
    for prop in numeric_props:
        # regex for e.g. hardness > 200 or hardness >= 200
        pattern = rf"{prop}\s*(>=|<=|=|>|<)\s*(\d+\.?\d*)"
        match = re.search(pattern, query)
        if match:
            op, val = match.groups()
            filters[prop] = f"{op}{val}"

    # Detect high/low keywords for properties
    for prop in numeric_props:
        if f"highest {prop}" in query or f"high {prop}" in query:
            filters[prop] = "high"
        elif f"lowest {prop}" in query or f"low {prop}" in query:
            filters[prop] = "low"

    return filters

def apply_numeric_filter(df, col, filter_value):
    """
    Applies a numeric filter string like '>200' or 'high' on df[col].
    Returns filtered df.
    """
    if filter_value == "high":
        return df.sort_values(by=col, ascending=False)
    elif filter_value == "low":
        return df.sort_values(by=col, ascending=True)
    else:
        # parse operator and value
        match = re.match(r"(>=|<=|=|>|<)(\d+\.?\d*)", filter_value)
        if not match:
            return df  # no filtering if invalid
        op, val_str = match.groups()
        val = float(val_str)
        if op == ">":
            return df[df[col] > val]
        elif op == "<":
            return df[df[col] < val]
        elif op == ">=":
            return df[df[col] >= val]
        elif op == "<=":
            return df[df[col] <= val]
        elif op == "=":
            return df[df[col] == val]
        else:
            return df

def filter_all_datasets(datasets, queries, top_n=10):
    """
    datasets: list of tuples (df, dataset_name)
    queries: dict of property filters, e.g. {"phase": "fcc", "hardness": ">200"}
    Returns: dict {dataset_name: filtered_df}
    """
    results = {}

    for df, name in datasets:
        df_filtered = df.copy()

        # First apply phase filter if present
        phase_filter = queries.get("phase", None)
        if phase_filter:
            # Find phase-like column
            phase_col = None
            for col in df_filtered.columns:
                if any(phase_key in col.lower() for phase_key in property_synonyms["phase"]):
                    phase_col = col
                    break
            if phase_col and df_filtered[phase_col].dtype == object:
                df_filtered[phase_col] = df_filtered[phase_col].str.strip()
                df_filtered = df_filtered[df_filtered[phase_col].str.contains(phase_filter, case=False, na=False)]
            else:
                # No phase column found or not applicable
                continue  # skip dataset if phase filter requested but no phase column found

        # Apply other property filters
        # Skip 'phase' key since handled above
        for prop, filter_val in queries.items():
            if prop == "phase":
                continue

            col = find_column_for_property(df_filtered, prop)
            if col is None or col not in df_filtered.columns:
                # Property column not found; skip this dataset as per your requirement
                df_filtered = None
                break

            # Drop rows with NaN in this column before filtering
            df_filtered = df_filtered[df_filtered[col].notna()]

            # Apply filter or sort
            df_filtered = apply_numeric_filter(df_filtered, col, filter_val)
            if df_filtered is None or df_filtered.empty:
                break  # no matching rows, break early

        if df_filtered is None or df_filtered.empty:
            continue  # no results for this dataset

        # Select columns to show: always formula + filtered property columns + phase (if any)
        show_cols = []
        if "formula" in df_filtered.columns:
            show_cols.append("formula")

        # Add filtered property columns found
        for prop in queries.keys():
            if prop == "phase":
                continue
            col = find_column_for_property(df_filtered, prop)
            if col and col in df_filtered.columns and col not in show_cols:
                show_cols.append(col)

        # Add phase column if phase filter applied
        if phase_filter and phase_col and phase_col not in show_cols:
            show_cols.append(phase_col)

        show_cols = [col for col in show_cols if col in df_filtered.columns]

        # For high/low filters, limit to top_n rows after sorting in apply_numeric_filter
        # For numeric filters with comparisons, just limit to top_n results anyway
        df_filtered = df_filtered[show_cols].head(top_n).copy()
        df_filtered["Source"] = name

        results[name] = df_filtered

    return results
import re

import pandas as pd

# Property synonyms map

property_synonyms = {

    "hardness": ["hv", "hardness", "vicker's hardness", "vickers hardness"],

    "bulk modulus": ["d_bulk (gpa)", "bulk modulus", "bulk_modulus", "bulk modulus"],

    "yield_strength": ["ys", "yield stress", "yield strength"],

    "ultimate_strength": ["uts", "tensile strength", "ultimate tensile strength"],

    "phase": ["phase_label", "bcc/fcc/other", "phase", "microstructure"],

    "density": ["density_exp", "density_calc", "density"]

}

def find_column_for_property(df, property_name):

    synonyms = property_synonyms.get(property_name.lower(), [property_name])

    for syn in synonyms:

        for col in df.columns:

            if syn.lower() in col.lower():

                return col

    return None

def parse_query_to_filters(question):

    """

    Parse question to a dict of property filters.

    Supports phase filters, numeric comparisons, and high/low keywords.

    """

    query = question.lower()

    filters = {}

    # Detect phase filter (fcc, bcc, hcp, other)

    for phase in ["fcc", "bcc", "hcp", "other"]:

        if phase in query:

            filters["phase"] = phase

            break

    # Detect numeric filters (hardness > 200 etc.)

    numeric_props = ["hardness", "bulk modulus", "yield strength", "ultimate strength", "density"]

    for prop in numeric_props:

        # regex for e.g. hardness > 200 or hardness >= 200

        pattern = rf"{prop}\s*(>=|<=|=|>|<)\s*(\d+\.?\d*)"

        match = re.search(pattern, query)

        if match:

            op, val = match.groups()

            filters[prop] = f"{op}{val}"

    # Detect high/low keywords for properties

    for prop in numeric_props:

        if f"highest {prop}" in query or f"high {prop}" in query:

            filters[prop] = "high"

        elif f"lowest {prop}" in query or f"low {prop}" in query:

            filters[prop] = "low"

    return filters

def apply_numeric_filter(df, col, filter_value):

    """

    Applies a numeric filter string like '>200' or 'high' on df[col].

    Returns filtered df.

    """

    if filter_value == "high":

        return df.sort_values(by=col, ascending=False)

    elif filter_value == "low":

        return df.sort_values(by=col, ascending=True)

    else:

        # parse operator and value

        match = re.match(r"(>=|<=|=|>|<)(\d+\.?\d*)", filter_value)

        if not match:

            return df  # no filtering if invalid

        op, val_str = match.groups()

        val = float(val_str)

        if op == ">":

            return df[df[col] > val]

        elif op == "<":

            return df[df[col] < val]

        elif op == ">=":

            return df[df[col] >= val]

        elif op == "<=":

            return df[df[col] <= val]

        elif op == "=":

            return df[df[col] == val]

        else:

            return df

def filter_all_datasets(datasets, queries, top_n=10):

    """

    datasets: list of tuples (df, dataset_name)

    queries: dict of property filters, e.g. {"phase": "fcc", "hardness": ">200"}

    Returns: dict {dataset_name: filtered_df}

    """

    results = {}

    for df, name in datasets:

        df_filtered = df.copy()

        # First apply phase filter if present

        phase_filter = queries.get("phase", None)

        if phase_filter:

            # Find phase-like column

            phase_col = None

            for col in df_filtered.columns:

                if any(phase_key in col.lower() for phase_key in property_synonyms["phase"]):

                    phase_col = col

                    break

            if phase_col and df_filtered[phase_col].dtype == object:

                df_filtered[phase_col] = df_filtered[phase_col].str.strip()

                df_filtered = df_filtered[df_filtered[phase_col].str.contains(phase_filter, case=False, na=False)]

            else:

                # No phase column found or not applicable

                continue  # skip dataset if phase filter requested but no phase column found

        # Apply other property filters

        # Skip 'phase' key since handled above

        for prop, filter_val in queries.items():

            if prop == "phase":

                continue

            col = find_column_for_property(df_filtered, prop)

            if col is None or col not in df_filtered.columns:

                # Property column not found; skip this dataset as per your requirement

                df_filtered = None

                break

            # Drop rows with NaN in this column before filtering

            df_filtered = df_filtered[df_filtered[col].notna()]

            # Apply filter or sort

            df_filtered = apply_numeric_filter(df_filtered, col, filter_val)

            if df_filtered is None or df_filtered.empty:

                break  # no matching rows, break early

        if df_filtered is None or df_filtered.empty:

            continue  # no results for this dataset

        # Select columns to show: always formula + filtered property columns + phase (if any)

        show_cols = []

        if "formula" in df_filtered.columns:

            show_cols.append("formula")

        # Add filtered property columns found

        for prop in queries.keys():

            if prop == "phase":

                continue

            col = find_column_for_property(df_filtered, prop)

            if col and col in df_filtered.columns and col not in show_cols:

                show_cols.append(col)

        # Add phase column if phase filter applied

        if phase_filter and phase_col and phase_col not in show_cols:

            show_cols.append(phase_col)

        show_cols = [col for col in show_cols if col in df_filtered.columns]

        # For high/low filters, limit to top_n rows after sorting in apply_numeric_filter

        # For numeric filters with comparisons, just limit to top_n results anyway

        df_filtered = df_filtered[show_cols].head(top_n).copy()

        df_filtered["Source"] = name

        results[name] = df_filtered

    return results


# Now the main integrated querying function

def query_hea_intelligent(question, top_k=5):

    print("query intel")

    faiss_results = faiss_index.similarity_search(question, k=top_k)

    faiss_text = "\n".join([doc.page_content for doc in faiss_results])

    print("faiss text done")

    # Parse question into structured property filters

    query_props = parse_query_to_filters(question)

    print(f"Parsed query filters: {query_props}")

    # Run flexible CSV filters across datasets

    csv_results_dict = filter_all_datasets(

        datasets=[(d1, "MPEA"), (d2, "MLPred"), (d3, "Achief")],

        queries=query_props,

        top_n=top_k

    )

    print("csv result collected")

    # Format CSV context for prompt

    csv_context = ""

    for name, df_filtered in csv_results_dict.items():

        csv_context += f"\n### {name} matches:\n{df_filtered.to_string(index=False)}\n"

    prompt = f"""

You are a materials scientist. Based on the following context, answer the question precisely, listing formulas and requested properties. Do NOT output multiple-choice questions.

FAISS context (from papers):

{faiss_text}

CSV datasets context:

{csv_context}

Question: {question}

Answer:

"""

    output = llm_pipe(prompt, max_length=512)[0]["generated_text"]

    return output, csv_results_dict, faiss_text




In [ ]:
# ✅ Manual test for query_hea_intelligent without Gradio
test_question = "Which HEAs have FCC structure and hardness > 200"

answer, csv_results, faiss_context = query_hea_intelligent(test_question)

print("🔹 LLM Answer:")
print(answer)

print("\n🔹 Filtered CSV Matches:")
for name, df in csv_results.items():
    print(f"\n--- {name} ---")
    display(df)  # Nice display in Colab

print("\n🔹 Paper Context (FAISS):")
print(faiss_context[:1000] + "...")


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


query intel
faiss text done
Parsed query filters: {'phase': 'fcc', 'hardness': '>200'}
csv result collected
🔹 LLM Answer:


You are a materials scientist. Based on the following context, answer the question precisely, listing formulas and requested properties. Do NOT output multiple-choice questions.
 
FAISS context (from papers):

107398908.pdf - conclusion: of an FCC phase, (C) lattice constants of a BCC phase
. . . . . . . . . . . . . . . .
9
1.10 Hardness for HEAs are compared with the conventional alloys including 17-4 PH
stainless steel, Hastealloy, and 316 stainless steel
. . . . . . . . . . . . . . . . . . . .
9
1.11 Yield strength and density property space chart. Newly discovered HEAs are super-
imposed on the chart with the conventional alloys. The dashed lines give performance
8605305.pdf - conclusion: in B0.7 sample 
143 
Figure 6.10 
Distribution of hardness values of FCC, Fe2B and BCC/B2 phases 
in B1.0 sample 
143
127093088.pdf - methods: tively. Moreover, in this study

,formula,HV,microstructure,Source
1,Al0.5 Co1 Fe1 Ni1,212.0,FCC+BCC,MPEA
2,Al0.75 Co1 Fe1 Ni1,385.0,FCC+BCC,MPEA
7,Co1 Fe1 Ni1 Si0.5,287.0,FCC+Sec.,MPEA
8,Co1 Fe1 Ni1 Si0.75,570.0,FCC+Sec.,MPEA
27,Al0.5 Co1 Cr1 Fe1 Ni1,209.0,FCC+BCC,MPEA



🔹 Paper Context (FAISS):
107398908.pdf - conclusion: of an FCC phase, (C) lattice constants of a BCC phase
. . . . . . . . . . . . . . . .
9
1.10 Hardness for HEAs are compared with the conventional alloys including 17-4 PH
stainless steel, Hastealloy, and 316 stainless steel
. . . . . . . . . . . . . . . . . . . .
9
1.11 Yield strength and density property space chart. Newly discovered HEAs are super-
imposed on the chart with the conventional alloys. The dashed lines give performance
8605305.pdf - conclusion: in B0.7 sample 
143 
Figure 6.10 
Distribution of hardness values of FCC, Fe2B and BCC/B2 phases 
in B1.0 sample 
143
127093088.pdf - methods: tively. Moreover, in this study it was demonstrated that some non-equiatomic HEAs
(FCC or BCC) have superior hardness compared to their equiatomic counterparts. For
instance, Fe32Ni12Cu27Co29 HEA has a 236 HV while Fe25Ni25Cu25Co25 HEA has 184 HV
for the FCC family and Cr18Mo32Nb35Ti15 has 607 HV while Cr25Mo25Nb25Ti25 has hard-
ness val

In [ ]:
!pip install -q gradio

In [ ]:
import gradio as gr
import pandas as pd

def gradio_query(question):
    answer, csv_results_dict, faiss_text = query_hea_intelligent(question)

    # Combine CSV results dict into one DataFrame with Source column
    if csv_results_dict:
        merged_df = pd.concat([df.assign(Source=name) for name, df in csv_results_dict.items()], ignore_index=True)
    else:
        merged_df = pd.DataFrame()  # empty if no matches

    # Return FAISS context first, then CSV matches table, then only the raw LLM answer
    # But gr.Interface outputs are in order: answer, table, context
    # So swap order: return (answer, merged_df, faiss_text)
    # To display FAISS first, CSV next, LLM last, rearrange output in gr.Interface outputs list below

    return answer, merged_df, faiss_text

demo = gr.Interface(
    fn=gradio_query,
    inputs=gr.Textbox(lines=2, placeholder="Ask about HEAs..."),
    outputs=[
        gr.Textbox(label="LLM Answer"),  # show FAISS context first
        gr.Dataframe(label="CSV Matches"),          # then CSV filtered table
        gr.Textbox(label="Paper Context (FAISS)")               # then answer only
    ],
    title="🔬 HEA Query",
    description="Query 3000+ HEA papers (via FAISS) + structured datasets (hardness, strength, phase, etc)."
)
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d3b764efe896368f65.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
